In [ ]:
# 필요한 기능 정의
import torch
import random
import time

class Timer:
    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        end_time = time.time()
        elapsed_time = end_time - self.start_time
        print(f"Elapsed time: {elapsed_time:.5f} seconds")



arr : 파이썬의 **기본** 리스트 **자료구조** 

tensor : pytorch의 **행렬 연산**을 하기 위한 자료구조

In [ ]:
arr = [ [1,2,3], [4,5,6], [7,8,9] ]
tensor = torch.tensor([ [1,2,3], [4,5,6], [7,8,9] ], dtype = torch.float32)

print(arr)
print(tensor, "\n", tensor.t())

### 파이썬 List의 대규모 내적 계산 (CPU 사용)

VS

### Pytorch Tensor의 대규모 내적 계산 (CPU 사용)

In [ ]:

# 일반 파이썬 내적 (O^3)
def naive_A_AT(A):
    n = len(A)
    return [
        [sum(A[i][k] * A[j][k] for k in range(n)) for j in range(n)]
        for i in range(n)
    ]

# 300 x 300 크기의 2차원 리스트 (일반 Python 리스트)
arr = [[random.random() for _ in range(300)] for _ in range(300)]
# 300 x 300 크기의 2차원 tensor, 난수 범위는 평균(μ)=0, 표준편차(σ)=1인 정규 분포(Gaussian Distribution)
tensor = torch.randn(300, 300)    

# 각각 자기자신 @ 자기자신.T 내적 시작
with Timer():
    print("파이썬 기본 리스트 자료구조의 내적 (CPU)")
    arr_dot = naive_A_AT(arr)

# 내부 C/C++ 로 인터페이스 전달 -> CPU 사용하지만 벡터화(vectorization), 멀티스레딩 활용
with Timer():
    print("파이토치 Tensor 타입의 내적 (CPU)")
    tensor_dot = tensor @ tensor.t()

### torch의 행렬 연산을 GPU를 사용한 결과

In [ ]:
# 단, 하드웨어 특성상 행렬 크기가 작으면 CPU가 더 빠름
cpu_tensor = torch.randn(1000, 1000).to("cpu")
gpu_tensor = torch.randn(1000, 1000).to("cuda:0")

with Timer():
    print("파이토치 Tensor 타입의 내적 (CPU)")
    tensor_dot = cpu_tensor @ cpu_tensor.t()

with Timer():
    print("파이토치 Tensor 타입의 내적 (GPU)")
    tensor_dot = gpu_tensor @ gpu_tensor.t()

### tensor 자료구조의 내장 기능

In [ ]:
tensor = torch.randn(100, 100)

print("Shape:", tensor.shape) # 차원 수를 나타냄 Ex) (배치크기, 데이터 수 , 피쳐 수) 처럼
print("Data type:", tensor.dtype) # 타입, 기본값은 float.32 !
print("Device:", tensor.device) # 현재 Tensor가 상주하고있는 device
print("Dimensions:", tensor.ndim) # 텐서의 차원수


### Tensor의 차원 관리 방법

In [ ]:
# 학습 데이터의 배치 크기가 10, 배치당 유저수가 100, 유저당 특징 수가 50 이라고 하면 다음과 같은 행렬이 준비됨
tensor = torch.randn(10, 100, 50)

# .transpose 연산 (dim0, dim1) -> 특정 두 차원을 교환
t_tensor = tensor.transpose(0, 2)
print("0번째 차원과, 2번쨰 차원을 변경 -> ",t_tensor.shape, "\n\n")

# 활용 예시 - CNN ) (N, C, H, W) -> (N, H, W, C)
x = torch.randn(8, 3, 224, 224)   # 8장, 채널 3, 224x224
x_t = x.transpose(1, 3)          # (8, 224, 224, 3) 로 변경 -> RNN 모델의 입력 형식


In [ ]:
# .view 함수 -> 메모리 공유
# 100명의 유저가 있고, 유저당 50개의 특징이 있는데 이를 10개의 배치로 분할하고싶음
arr = [i for i in range(1,1000+1)]
tensor = torch.tensor(arr).to('cuda:0')
tensor = tensor.view(100,10).to('cuda:0')
tensor = tensor.view(10,10,10).to('cuda:0')

print(tensor)

In [ ]:
# .reshape 함수 -> 텐서를 복사
# 100명의 유저가 있고, 유저당 50개의 특징이 있는데 이를 10개의 배치로 분할하고싶음
arr = [i for i in range(1,1000+1)]
tensor = torch.tensor(arr).to('cuda:0')
tensor = tensor.reshape(100,10).to('cuda:0')
tensor = tensor.reshape(10,10,10).to('cuda:0')

print(tensor)

### squeeze 함수


-> 쓸때없는 1차원을 삭제해주는 유틸성 함수!


-> 매개변수로 원하는 차원 수를 받고, 이 해당 차원이 1일떄만 삭제, 그렇지 않으면 아무 동작 X

In [ ]:
# 3중 분류 MLP 모델에서 최종 출력층의 확률값이 다음과 같이 나왔다고 가정하자.
logits = torch.FloatTensor([0.25, 0.45, 0.30])
# 근데 여기서 레이블을 추출하기 위해 가장 높은 값의 인덱스를 알고싶다.
label = torch.argmax(logits)
print(f"1차원 tensor에서 label -> {label}","\n")

# 일반적으로 모델의 입/출력 형식이 다음과 같다.
logits = torch.FloatTensor(
    [
        [0.25],
        [0.45],
        [0.30]
    ]
)

print(f"모델 출력 Shape -> {logits.shape}")

# 근데 여기서 레이블을 추출하기 위해 가장 높은 값의 인덱스를 알고싶으면 차원을 제거해야한다.
squeezed_logits = logits.squeeze(1)
print(squeezed_logits, squeezed_logits.shape)
label = torch.argmax(squeezed_logits)
print(f"다차원 텐서에서 label -> {label}")

### Unsqueeze 함수


-> 쓸때없는 1차원을 추가해주는 유틸성 함수!

-> 매개변수로 1차원을 추가하고싶은 곳을 받음

In [ ]:
# 사람의 특징을 가지고있고, 이 사람의 키와 몸무게로 성별을 예측한다고 하자 (이진분류)
# 나는 특징 행렬값을 다음과 같이 들고 있다.

feature = torch.tensor([180,80])
print(f"나의 데이터 ->",feature, feature.shape)

# MLP 모델은 일반적으로 다음과 같은 형식으로 데이터를 받는다

mlp_input = torch.tensor(
    [
        [180], # 입력층의 첫 번째 뉴런으로 들어가는 값
        [80] # 입력층의 두 번째 뉴런으로 들어가는 값
    ]
)

# 그러면 내 데이터에 쓸때없는 1차원 을 추가해야한다. 이때 Unsqueeze를 쓰면 편하다
mlp_feature = feature.unsqueeze(1)
print("언스퀴즈 내 데이터 ->")
print(mlp_feature)
print("shape -> ",mlp_feature.shape)


### Mask 행렬 기능

In [ ]:
items_A = torch.tensor([0,0,1,1,0,0])
items_B = torch.tensor([0,0,0,0,0,0])

mask = (items_A == items_B)
print(mask)


### Tensor 자료구조의 Gradient값

In [ ]:
tensor = torch.randn(10,5, requires_grad=True)
print(tensor)

In [ ]:
import torch

x = torch.tensor([3.0], requires_grad=True)

y = 2*x + 5
y.backward()

print("x.grad =", x.grad) 

In [1]:
import torch
import torch.nn as nn

model = nn.Linear(20000, 10000)
x = torch.randn(10000, 20000, requires_grad=True)

# with torch.no_grad():
#     output_no_grad = model(x)

# print("no_grad() 블록 안에서의 forward 결과 -> ", output_no_grad)

output_grad = model(x)
print("no_grad() 블록 밖에서의 forward 결과 -> ", output_grad)

# 이제 실제 backward 실행
# output_grad는 requires_grad=True 이므로 기울기 계산 가능
loss = output_grad.sum()  # 임의의 스칼라 예시
loss.backward()

print()
print("x.grad =", x.grad)


no_grad() 블록 밖에서의 forward 결과 ->  tensor([[ 1.0870,  0.1964,  0.3528,  ..., -0.6517,  0.0755,  0.2146],
        [-0.5185, -1.1291,  1.1068,  ...,  0.9242, -0.0219, -0.2693],
        [-0.0414,  0.7751, -0.0735,  ..., -0.8630, -0.0411,  1.4447],
        ...,
        [ 0.4864,  0.8743, -0.1774,  ...,  0.1577, -0.0595, -0.8307],
        [ 0.2549,  0.2407, -0.4051,  ..., -0.0610,  0.1141, -1.1097],
        [-0.8600,  0.0898, -0.5356,  ..., -0.4654, -0.1645, -0.4457]],
       grad_fn=<AddmmBackward0>)

x.grad = tensor([[-0.5393, -0.5913,  0.0603,  ..., -0.4345, -0.9437,  0.1226],
        [-0.5393, -0.5913,  0.0603,  ..., -0.4345, -0.9437,  0.1226],
        [-0.5393, -0.5913,  0.0603,  ..., -0.4345, -0.9437,  0.1226],
        ...,
        [-0.5393, -0.5913,  0.0603,  ..., -0.4345, -0.9437,  0.1226],
        [-0.5393, -0.5913,  0.0603,  ..., -0.4345, -0.9437,  0.1226],
        [-0.5393, -0.5913,  0.0603,  ..., -0.4345, -0.9437,  0.1226]])
